# TCN text classification model
<hr>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
corpus = pd.read_csv("places_names_labelled.csv", index_col = 0)
# corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(25161, 8)


,name,clean_name,place_id,vicinity,no_of_ratings,avg_rating,type_1,purpose
0,Colombo Swimming Club,colombo swimming club,ChIJETFLQ0FZ4joRwioIp4MnzHA,148 “Storm Lodge” Galle Road 3,1181.0,4.4,tourist_attraction,recreational
1,Storm lodge,storm lodge,ChIJGSU_sGtZ4joRiyZjgglUlPE,"148 “Storm Lodge” Galle Road 3, Colombo 00300",NaN,NaN,lodging,personal
5,Ayura,ayura,ChIJQwy6bEFZ4joRU9m1ylxsbMs,"142, Yathama Building, Galle Road, Colombo",16.0,4.5,jewelry_store,shopping
6,Baby Gallery Kurulapina,baby gallery kurulapina,ChIJR1dqFUFZ4joRsqACvaqRNws,"WR8X+655, Colombo",NaN,NaN,clothing_store,shopping
7,"Galle Face , Taj Hotel",galle face taj hotel,ChIJo015E0FZ4joRdGUr8gWxo4E,"138 Colombo - Galle Main Road, Colombo",2.0,5.0,restaurant,dining
...,...,...,...,...,...,...,...,...
44395,Ichiro Motoring - Wijerama,ichiro motoring wijerama,ChIJk3QsSX1a4joRDO-HD30GZT8,"High Level Road, Nugegoda",60.0,3.5,car_repair,personal
44396,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44397,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44398,Mirai Auto Land (Pvt) Ltd,mirai auto land pvt ltd,ChIJ6V8_E2da4joREMxUiWN0uVA,"Delkanda, 383 High Level Rd, Avissawella Road,...",52.0,4.4,car_repair,personal


In [3]:
corpus['purpose'].replace(corpus['purpose'].unique(), list(range(0, len(corpus['purpose'].unique()))), inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,corpus['purpose'],test_size=0.2, random_state=32)

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>## Develop Vocabulary

A part of preparing text for text classification involves defining and tailoring the vocabulary of words supported by the model. **We can do this by loading all of the documents in the dataset and building a set of words.**

The larger the vocabulary, the more sparse the representation of each word or document. So, we may decide to support all of these words, or perhaps discard some. The final chosen vocabulary can then be saved to a file for later use, such as filtering words in new documents in the future.

In [6]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
# X_train,X_test,y_train,y_test
# Separate the sentences and the labels
train_x = list(X_train.clean_name)
train_y = np.array(X_train.purpose)
test_x = list(X_test.clean_name)
test_y = np.array(X_test.purpose)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  wycherley montessori
Into a sequence of int: [1398, 122]
Into a padded sequence: [1398  122    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]


In [8]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
ltd 2
pvt 3
lanka 4
the 5
sri 6
colombo 7
of 8
and 9
house 10
13138


## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [9]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(8, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [10]:
model_0 = define_model(input_dim=1000, max_length=100)
model_0.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d[0][0]          
_______________________________________________________________________________________

In [13]:
# tcn_full_summary(model_0)

In [11]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4]

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# # Separate the sentences and the labels
# train_x = list(corpus[corpus.split=='train'].sentence)
# train_y = np.array(corpus[corpus.split=='train'].label)
# test_x = list(corpus[corpus.split=='test'].sentence)
# test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the input shape
        model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.7):

            model.fit(Xtrain, train_y, batch_size=50, epochs=50, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            else:
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

-------------------------------------------
Training 1: tanh activation, 1 kernel size.
-------------------------------------------
Epoch 1/50
403/403 [==============================] - 49s 123ms/step - loss: 1.1872 - accuracy: 0.5882 - val_loss: 0.7973 - val_accuracy: 0.7461
Epoch 2/50
403/403 [==============================] - 49s 121ms/step - loss: 0.6163 - accuracy: 0.8115 - val_loss: 0.7590 - val_accuracy: 0.7598
Epoch 3/50
403/403 [==============================] - 47s 117ms/step - loss: 0.3940 - accuracy: 0.8810 - val_loss: 0.7835 - val_accuracy: 0.7421
Epoch 4/50
403/403 [==============================] - 50s 124ms/step - loss: 0.2913 - accuracy: 0.9131 - val_loss: 0.8390 - val_accuracy: 0.7363
Epoch 5/50
403/403 [==============================] - 50s 124ms/step - loss: 0.2251 - accuracy: 0.9340 - val_loss: 0.9005 - val_accuracy: 0.7546
Epoch 6/50
403/403 [==============================] - 53s 131ms/step - loss: 0.1887 - accuracy: 0.9445 - val_loss: 0.9465 - val_accuracy: 0.751

KeyboardInterrupt: 

## Summary

In [19]:
record.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
2,relu,3,0.900
9,tanh,4,0.898
3,relu,4,0.894
7,tanh,2,0.892
4,relu,5,0.890
1,relu,2,0.888
5,relu,6,0.886
0,relu,1,0.882
6,tanh,1,0.880
8,tanh,3,0.880


In [20]:
report = record.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC.xlsx', sheet_name='random')